# **Polyhedra**
$\DeclareMathOperator*{\conv}{conv}$
$\DeclareMathOperator*{\cone}{cone}$

$P = \left\{ x \middle| Ax \leq b \right\}$

$P = \conv( V ) + \cone( W )$

In [3]:
import numpy as np

## **Incremental algorithm**
In what it follows we implement the double description method

In [80]:
def double_description_method( V, k ) :
  [ m, n ] = V.shape
  I = []
  Jp = []
  Jn = []
  for l in range( 0, n ) :
    if V[k,l] == 0 :
      I.append( l )
    elif V[k,l] > 0 :
      Jp.append( l )
    elif V[k,l] < 0 :
      Jn.append( l )
      
  p = len( I )
  print( I )
  print( Jp )
  print( Jn )
  if p > 0 :
    W = V[:,I]

  if len( Jp ) > 0 and len( Jn ) == 0  :
    W = np.insert( W, np.repeat( p, len( Jp ) ), V[:,Jp], axis = 1 )
    W[k,:] = 0
    
  elif len( Jp ) == 0 and len( Jn ) > 0  :
    W = np.insert( W, np.repeat( p, len( Jn ) ), V[:,Jn], axis = 1 )
    W[k,:] = 0
    
  elif len( Jp ) > 0 and len( Jn ) > 0 :
    
    for i in Jp :
      for j in Jn :
        vki = V[k,i]
        vkj = V[k,j]
#         d = vki - vkj
        d = 1
        if d != 0 :
          w = ( vki / d ) * V[:,j] - ( vkj / d ) * V[:,i]
          if p > 0 :
            W = np.insert( W, p, w, axis = 1 )
            p = W.shape[1]
          else :
            W = w.T
            W.shape = [ m, 1 ]
            p = 1

  return W

In [81]:
d = 5
A = np.ones( [1,d] )
A = np.insert( A, 1, -np.eye( d ), axis = 0 )
print( A )
b = np.zeros( A.shape[0] )
b[0] = 1
print( b )

[[ 1.  1.  1.  1.  1.]
 [-1. -0. -0. -0. -0.]
 [-0. -1. -0. -0. -0.]
 [-0. -0. -1. -0. -0.]
 [-0. -0. -0. -1. -0.]
 [-0. -0. -0. -0. -1.]]
[1. 0. 0. 0. 0. 0.]


In [82]:
[ m, d ] = A.shape
W = np.insert( np.eye( d ), d, A, axis = 0 )
V = np.insert( W, np.repeat( d, d ), -W, axis = 1 )
W = np.insert( np.zeros( [ d, m ] ), d, np.eye( m ), axis = 0 )
V = np.insert( V, np.repeat( 2*d, m ), W, axis = 1 )
del W
print( V )


Vk = np.copy( V )
[ M, N ] = Vk.shape
for j in range( 0, N ) :
  Vk[range( d, M ),j] = Vk[range( d, M ),j] - b

[[ 1.  0.  0.  0.  0. -1. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0. -0. -1. -0. -0. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0. -0. -0. -1. -0. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0. -0. -0. -0. -1. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1. -0. -0. -0. -0. -1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1. -1. -1. -1. -1. -1.  1.  0.  0.  0.  0.  0.]
 [-1. -0. -0. -0. -0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [-0. -1. -0. -0. -0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [-0. -0. -1. -0. -0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [-0. -0. -0. -1. -0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.]
 [-0. -0. -0. -0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.]]


In [83]:
print( Vk )
for k in range( 0, m ) :
  Vk = double_description_method( Vk, d + k )
  print( d + k, ':\n', Vk )

[[ 1.  0.  0.  0.  0. -1. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0. -0. -1. -0. -0. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0. -0. -0. -1. -0. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0. -0. -0. -0. -1. -0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1. -0. -0. -0. -0. -1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -2. -2. -2. -2. -2.  0. -1. -1. -1. -1. -1.]
 [-1. -0. -0. -0. -0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [-0. -1. -0. -0. -0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [-0. -0. -1. -0. -0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [-0. -0. -0. -1. -0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.]
 [-0. -0. -0. -0. -1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.]]
[0, 1, 2, 3, 4, 10]
[]
[5, 6, 7, 8, 9, 11, 12, 13, 14, 15]
5 :
 [[ 1.  0.  0.  0.  0.  0. -1. -0. -0. -0. -0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0. -0. -1. -0. -0. -0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0. -0. -0. -1. -0. -0.  0.  0.  0.  0.  0